In [35]:
# Graph Search
def graph_search(problem):
    frontier = [(problem.initial_state(), [])]
    explored = set()

    while frontier:
        state, path = frontier.pop(0)

        if problem.goal_test(state):
            return path

        explored.add(state)

        for action, result_state in problem.successor_fn(state):
            if result_state not in explored:
                new_path = path + [action]
                frontier.append((result_state, new_path))

    return None

In [36]:
# Tree Search
class Node:
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost

    def __repr__(self):
        return f"<Node {self.state}>"

def tree_search(problem, frontier):
    start_node = Node(problem.initial_state)
    frontier.append(start_node)
    explored = set()

    while frontier:
        node = frontier.pop(0)
        if problem.is_goal(node.state):
            return node
        explored.add(node.state)

        for child_state, action, step_cost in problem.get_successors(node.state):
            if child_state not in explored:
                child_node = Node(child_state, node, action, node.path_cost + step_cost)
                frontier.append(child_node)

    return None

In [37]:
# DFS
from collections import deque

def breadth_first_search(graph, start_node, goal_node):
    queue = deque([(start_node, [])])  # Initialize the queue with the start node and an empty path
    explored = set()  # Set to keep track of explored nodes

    while queue:
        current_node, path = queue.popleft()  # Get the next node and its corresponding path from the queue

        if current_node == goal_node:
            return path  # Return the path if the goal node is reached

        explored.add(current_node)  # Add the current node to the explored set

        for neighbor in graph[current_node]:
            if neighbor not in explored:
                queue.append((neighbor, path + [neighbor]))  # Add neighbors to the queue with an updated path

    return None  # Return None if the goal node is not found

graph = {
    'A': ['B', 'C'],
    'B': ['D', 'E'],
    'C': ['F'],
    'D': [],
    'E': ['F'],
    'F': []
}

start_node = 'A'
goal_node = 'F'
path = breadth_first_search(graph, start_node, goal_node)
print("Path:", path)

Path: ['C', 'F']


In [38]:
def depth_first_search(graph, start_node, goal_node):
    stack = [(start_node, [])]  # Initialize the stack with the start node and an empty path
    explored = set()  # Set to keep track of explored nodes

    while stack:
        current_node, path = stack.pop()  # Get the next node and its corresponding path from the stack

        if current_node == goal_node:
            return path  # Return the path if the goal node is reached

        explored.add(current_node)  # Add the current node to the explored set

        for neighbor in graph[current_node]:
            if neighbor not in explored:
                stack.append((neighbor, path + [neighbor]))  # Add neighbors to the stack with an updated path

    return None  # Return None if the goal node is not found

graph = {
    'A': ['B', 'C'],
    'B': ['D', 'E'],
    'C': ['F'],
    'D': [],
    'E': ['F'],
    'F': []
}
start_node = 'A'
goal_node = 'F'
path = depth_first_search(graph, start_node, goal_node)
print("Path:", path)

Path: ['C', 'F']


In [39]:
def depth_limited_search(graph, start_node, goal_node, depth_limit):
    return recursive_dls(graph, start_node, goal_node, depth_limit)

def recursive_dls(graph, current_node, goal_node, depth_limit):
    if current_node == goal_node:
        return [current_node]

    if depth_limit == 0:
        return []

    if depth_limit > 0:
        for neighbor in graph[current_node]:
            path = recursive_dls(graph, neighbor, goal_node, depth_limit - 1)
            if path:
                return [current_node] + path

    return []

graph = {
    'A': ['B', 'C'],
    'B': ['D', 'E'],
    'C': ['F'],
    'D': [],
    'E': ['F'],
    'F': []
}

start_node = 'A'
goal_node = 'F'
depth_limit = 3
path = depth_limited_search(graph, start_node, goal_node, depth_limit)
print("Path:", path)

Path: ['A', 'B', 'E', 'F']


In [40]:
# UCS (Uniform Cost Search)
from queue import PriorityQueue

def uniform_cost_search(graph, start_node, goal_node):
    frontier = PriorityQueue()  # Priority queue to store nodes based on their path cost
    frontier.put((0, start_node))  # Add the start node to the frontier with a priority of 0
    explored = set()  # Set to keep track of explored nodes
    path_cost = {start_node: (0, None)}  # Dictionary to store the path cost and parent node of each node

    while not frontier.empty():
        _, current_node = frontier.get()  # Get the node with the lowest path cost from the frontier

        if current_node == goal_node:
            return reconstruct_path(start_node, goal_node, path_cost)  # Return the path if the goal node is reached

        explored.add(current_node)  # Add the current node to the explored set

        for neighbor, cost in graph[current_node].items():
            new_cost = path_cost[current_node][0] + cost

            if neighbor not in explored and (neighbor not in path_cost or new_cost < path_cost[neighbor][0]):
                path_cost[neighbor] = (new_cost, current_node)
                frontier.put((new_cost, neighbor))  # Add the neighbor to the frontier with its updated path cost

    return None  # Return None if the goal node is not found

def reconstruct_path(start_node, goal_node, path_cost):
    path = [goal_node]
    current_node = goal_node

    while current_node != start_node:
        current_node = path_cost[current_node][1]  # Get the parent node of the current node
        path.append(current_node)

    path.reverse()
    return path


# Example usage:
graph = {
    'A': {'B': 5, 'C': 3},
    'B': {'D': 2, 'E': 4},
    'C': {'F': 6},
    'D': {'G': 1},
    'E': {'F': 3},
    'F': {'G': 2},
    'G': {}
}

start_node = 'A'
goal_node = 'G'
path = uniform_cost_search(graph, start_node, goal_node)
print("Path:", path)

Path: ['A', 'B', 'D', 'G']


In [41]:
# IDDFS (Iterative deepening depth-first search algorithm)
def iterative_deepening_dfs(graph, start_node, goal_node):
    depth_limit = 0  # Initialize the depth limit

    while True:
        result = depth_limited_dfs(graph, start_node, goal_node, depth_limit)
        if result is not None:
            return result  # Return the result if a path is found
        depth_limit += 1  # Increase the depth limit if a path is not found

    return None  # Return None if no path is found within the depth limit


def depth_limited_dfs(graph, current_node, goal_node, depth_limit, depth=0, path=[]):
    if current_node == goal_node:
        return path + [current_node]  # Return the path if the goal node is reached

    if depth == depth_limit:
        return None  # Return None if the depth limit is reached

    if depth < depth_limit:
        path = path + [current_node]  # Append the current node to the path

        for neighbor in graph[current_node]:
            result = depth_limited_dfs(graph, neighbor, goal_node, depth_limit, depth + 1, path)

            if result is not None:
                return result  # Return the result if a path is found

    return None  # Return None if no path is found

graph = {
    'A': ['B', 'C'],
    'B': ['D', 'E'],
    'C': ['F'],
    'D': [],
    'E': [],
    'F': ['G'],
    'G': []
}

start_node = 'A'
goal_node = 'G'
path = iterative_deepening_dfs(graph, start_node, goal_node)
print("Path:", path)

Path: ['A', 'C', 'F', 'G']


In [42]:
# BDS (Bi-Directional search algorithm)
from collections import deque

def bidirectional_search(graph, start_node, goal_node):
    forward_queue = deque([start_node])  # Queue for forward search
    backward_queue = deque([goal_node])  # Queue for backward search
    forward_visited = set([start_node])  # Set to store visited nodes for forward search
    backward_visited = set([goal_node])  # Set to store visited nodes for backward search
    forward_parent = {start_node: None}  # Dictionary to store parent nodes for forward search
    backward_parent = {goal_node: None}  # Dictionary to store parent nodes for backward search

    intersect_node = None  # Common node between forward and backward search

    while forward_queue and backward_queue:
        # Perform forward search
        forward_node = forward_queue.popleft()

        # Check if forward search intersects with backward search
        if forward_node in backward_visited:
            intersect_node = forward_node
            break

        # Add unvisited neighboring nodes to forward queue
        for neighbor in graph[forward_node]:
            if neighbor not in forward_visited:
                forward_queue.append(neighbor)
                forward_visited.add(neighbor)
                forward_parent[neighbor] = forward_node

        # Perform backward search
        backward_node = backward_queue.popleft()

        # Check if backward search intersects with forward search
        if backward_node in forward_visited:
            intersect_node = backward_node
            break

        # Add unvisited neighboring nodes to backward queue
        for neighbor in graph[backward_node]:
            if neighbor not in backward_visited:
                backward_queue.append(neighbor)
                backward_visited.add(neighbor)
                backward_parent[neighbor] = backward_node

    # If there is an intersection node, construct the path
    if intersect_node is not None:
        forward_path = reconstruct_path(start_node, intersect_node, forward_parent)
        backward_path = reconstruct_path(goal_node, intersect_node, backward_parent)
        backward_path.reverse()
        return forward_path + backward_path

    return None  # Return None if no path is found


def reconstruct_path(start_node, goal_node, parent):
    path = []
    current_node = goal_node

    while current_node != start_node:
        path.append(current_node)
        current_node = parent[current_node]

    path.append(start_node)
    path.reverse()
    return path

graph = {
    'A': {'B': 1, 'C': 2},
    'B': {'A': 1, 'D': 3},
    'C': {'A': 2, 'D': 1},
    'D': {'B': 3, 'C': 1, 'E': 2},
    'E': {'D': 2, 'F': 3},
    'F': {'E': 3, 'G': 2},
    'G': {'F': 2}
}

start_node = 'A'
goal_node = 'G'
path = bidirectional_search(graph, start_node, goal_node)
print("Path:", path)

Path: ['A', 'B', 'D', 'D', 'E', 'F', 'G']


In [43]:
from queue import PriorityQueue

def greedy_bfs(graph, start_node, goal_node, heuristic):
    visited = set()
    queue = PriorityQueue()
    queue.put((heuristic[start_node], start_node))  # Enqueue start node with heuristic value

    while not queue.empty():
        _, current_node = queue.get()

        if current_node == goal_node:
            # Goal node found, return the path
            return get_path(graph, start_node, goal_node)

        visited.add(current_node)

        for neighbor in graph[current_node]:
            if neighbor not in visited:
                queue.put((heuristic[neighbor], neighbor))

    return None  # No path found


def get_path(graph, start_node, goal_node):
    path = []
    current_node = goal_node

    while current_node != start_node:
        path.append(current_node)
        current_node = next(iter(graph[current_node]))

    path.append(start_node)
    path.reverse()
    return path


graph = {
    'A': {'B', 'C'},
    'B': {'A', 'D'},
    'C': {'A', 'E'},
    'D': {'B', 'F'},
    'E': {'C', 'G'},
    'F': {'D'},
    'G': {'E', 'H'},
    'H': {'G'}
}

heuristic = {
    'A': 8,
    'B': 6,
    'C': 4,
    'D': 4,
    'E': 6,
    'F': 2,
    'G': 4,
    'H': 0
}

start_node = 'A'
goal_node = 'H'
path = greedy_bfs(graph, start_node, goal_node, heuristic)
print("Path:", path)


Path: ['A', 'C', 'E', 'G', 'H']


In [44]:
# A* Search Algorithm
from queue import PriorityQueue

def a_star_search(graph, start_node, goal_node, heuristic):
    visited = set()
    queue = PriorityQueue()
    queue.put((0, start_node, [start_node]))  # Enqueue start node with initial cost and path

    while not queue.empty():
        _, current_node, path = queue.get()

        if current_node == goal_node:
            # Goal node found, return the path
            return path

        if current_node not in visited:
            visited.add(current_node)

            for neighbor, cost in graph[current_node].items():
                if neighbor not in visited:
                    total_cost = len(path) + cost + heuristic[neighbor]
                    queue.put((total_cost, neighbor, path + [neighbor]))

    return None  # No path found


graph = {
    'A': {'B': 5, 'C': 3},
    'B': {'D': 2, 'E': 4},
    'C': {'F': 6},
    'D': {'G': 1},
    'E': {'G': 3},
    'F': {'G': 7},
    'G': {}
}

heuristic = {
    'A': 7,
    'B': 4,
    'C': 2,
    'D': 1,
    'E': 4,
    'F': 5,
    'G': 0
}

start_node = 'A'
goal_node = 'G'
path = a_star_search(graph, start_node, goal_node, heuristic)
print("Path:", path)

Path: ['A', 'B', 'D', 'G']
